# Pluralsite ML Coding Challenge
Daniel Stack, dstack1776@gmail.com 

github handle: dstack1776

June 23, 2018

# Instructions

There are two separate Python files. They should be put in the same directory. The first of these is "Pluralsight Class Modules". I have provided it both as a Jupyter Notebook and as a standard Python file. The function main makes a number of calls to parse the csv files, calculate similarity, and create an sqlite database "dstack.db" that stores the results in the table distance_matrix, with three columns - src_usr, dst_usr, and distance. This takes several minutes to complete - there's some optimizations I'd consider (and will discuss below) were this not a model but rather a much larger dataset.

The second file is rest_dstack.py that serves as a RESTful API interface. Like the first program, it is a toy with basic functionality. In this case is is listening to 127.0.0.1:5002. There are two get commands, users and usersn, detailed below.

## users
"users" has a single argument, the user handle, which ranges from 1 to 10000, inclusive. It returns a dictionary of the ten nearest user handles (not including itself) and the distance, which ranges from 0 (identical) to 1 (nothing in common). For example, 127.0.0.1:5002/users/9999 will return the ten nearest handles to user handle 9999 as well as the distances from 9999. I ran this program straight out of IDLE. It needs to be run after "Pluralsight Class Modules" as it does a simple sql select from dstack.db.

Example output for 127.0.0.1:5002/users/9999

```python
{"1562": 0.0, "5238": 0.0, "6303": 0.0, "4484": 0.5, "4432": 0.5196152422706632, "906": 0.5204164998665332, "2200": 0.5443310539518175, "3605": 0.5446711546122731, "5071": 0.5608545472127794, "453": 0.5773502691896258}
```

## usersn
"usersn" has two arguments, id and num. id is used for the user handle while num indicates how many handles should be returned, in ascending order, starting with the closest.

Example output for http://127.0.0.1:5002/usersn?id=9999&num=15
```python
{"1562": 0.0, "5238": 0.0, "6303": 0.0, "4484": 0.5, "4432": 0.5196152422706632, "906": 0.5204164998665332, "2200": 0.5443310539518175, "3605": 0.5446711546122731, "5071": 0.5608545472127794, "453": 0.5773502691896258, "637": 0.5773502691896258, "2754": 0.5773502691896258, "5411": 0.5773502691896258, "6188": 0.5773502691896258, "7688": 0.5773502691896258}
```


# Similarity Calculation
## Aborted Effort
When I first looked into this problem, I had initially considered using cosine similarity for the User Assessments. With all of the assessments having numeric values it seemed a reasonable mechanism. However, this entailed creating a column for every assessment type and the code to an **extremely** long time to run, even with this limited dataset. Moreover, it became clear that even were the performance model to be overcome, it was not a good model. There were users who took no assessments - which made them identical to other users who took no assessments, as I needed to give them some numeric value. In the end, this did not appear a good model to use.

## Three-Axis Jaccard Distance
Instead, I made use of three axes of Jaccard distance. I set up an axis for assessments, classes examined, and self-described interests.

Jaccard similarity is used to compare sets. It is the cardinality of the intersection of two sets divided by the union of the two sets. I added some code to make the similarity between empty sets as 0 (so as to avoid divide by zero error). Given the most similar (identical) would have a 1 under this model, I subtracted the result from 1 to get a distance.

I settled on making use of a set compare given the wide variety of interests, courses, and assessments. This avoided hoops I ran into when attempting to make use of cosine similarity.

### Assessments
One thing I did not want to lose on assessments were the scores. I wanted similarity between people who took assessments in a given are but I wanted even greater similarity if their level of experience were similar. To do this I added three possible tags to every assessment - Novice, Proficient, and Expert. Someone who tested as a Novice in Python, for example, would have the entry 'Python-Novice' added to their assessment set. Someone who tested as an Expert in Python have 'Python-Novice', 'Python-Proficient', and 'Python-Expert; added to their assessment set. In this model, two Python Experts would have a strong level of similarity given they would have three matching entries in their sets. Similarly a Python Novice and Python Expert would still have some level of similarity. I used benchmarks of 100 and 200 to show Proficiency and Expertise, aligning with Pluralsight's testing algorithms.

### Interests
Interests were the most straightforward as they were simple binaries - a user was either interested in something or not. I added interests into a user's Interest set.

### Classes
Given the variety of courses, I took advantage of the course tags to simplify areas of interest into broader categories - for each user, instead of using the course name I instead mapped in the tags from the separate csv file, using logic akin to a SQL left join. Like the Assessments, I took advantage of the difficulty of the course. I did not use the time spent on a class's webpage, an area of potential enhancement - perhaps using it to measure an intensity of interest - or to possibly discard entries that a user spent an extremely brief amount of time on.

## Combining the Three Axes
Since I calculated three separate distance axes, I combined them using a simple application of the Pythagorean Theorem, and to keep the result scaled from 0 to 1, I divided this result by the square root of 3. 


# Considerations at Larger Scale

I was able to take advantage of storing a lot of data in memory. Indeed, some of my initial efforts involved building a Jaccard Distance table for everything. It would have been possible to go with this model and not used a database table at all. However, the database table has as an advantage it can be built over time and in pieces. For example, I built the SQL table for one user at a time, comparing it with all potential other users in one 1D array. This was a large data structure - at a larger scale it might be reasonable (or even necessary) to build it in smaller chunks. Conversely, I performed and committed the SQL writes one at a time - by use of transactions - http://bytes.schibsted.com/speeding-up-sqlite-insert-operations/ - it becomes possible to speed up that process. 

To speed up calculations it is possible to take advantage of the distance between a source and destination is commutative - you only need to calculate it once.

# Other Thoughts

One thing I would have liked would have been some information as to what sorts of assessments, class tags, and interests are similar to one another. Domain knowledge could have been used to make such decisions, likely codifying it with a CSV file similar to the classes' tag file.  For example, Pluralsight's webpage breaks the assessments into Development, IT Ops, Data, Security, Creative. These may have been data points appropriate for similarity calculations.

The REST interface is fairly basic and a good opportunity for improvement. This was my first exercise at creating a REST interface - since on my interview with Connor I spoke how I have talent at developing new skills, it seemed an absolutely fair challenge for me to illustrate my capacity to do just that.

In [1]:
import numpy as np
import pandas as pd
import array
import sqlite3


In [2]:
conn = sqlite3.connect('dstack.db')


In [3]:
def jaccard_distances_array(in_array, num_users):
    """
    Create an array of Jaccard distances for N users by N users - in the sample set this would
    be 10,000 by 10,000. In the end I did not use this method for my final design but am
    maintaining it for the time being.
    
    Inputs - 
    in_array - a 1D array w/ an entry per user. The entry is a set for Jaccard distance calcualtion.
    num_users - Number of users
    
    Returns - A Jaccard distance array
    """

    distance_sets = np.full((num_users, num_users),1.0)

    for src_index, src_row in enumerate(distance_sets):

        for dst_index, dst_row in enumerate(in_array):
            # do a triangle and mirror
            if src_index > dst_index:
                continue
            src_row = in_array[src_index]
            dst_row = in_array[dst_index]
            if src_row and dst_row:
                # if either set is empty no point continuing. Otherwise get cardinalities
                intersection_cardinality = len(set.intersection(*[set(src_row), set(dst_row)]))
                union_cardinality = len(set.union(*[set(src_row), set(dst_row)]))
                distance_sets[src_index][dst_index] = 1.0 - (
                    float(intersection_cardinality)/float(union_cardinality))
                
    for src_index, src_row in enumerate(distance_sets):
        for dst_index, dst_row in enumerate(distance_sets):
            # mirror half of the triangle
            if src_index > dst_index:
                distance_sets[src_index][dst_index] = distance_sets[dst_index][src_index]
        print(distance_sets[src_index])
    return distance_sets

In [4]:

def jaccard_distances_user(in_array, user_handle, num_users):
    """
    For a given user_handle, calculate the distance to all other users. 
    In the end I did not use this method for my final design but am
    maintaining it for the time being.
    
    Inputs - 
    in_array - a 1D array w/ an entry per user. The entry is a set for Jaccard distance calcualtion.
    user_handle - 1-based user handle
    num_users - Number of users
    
    Returns - A Jaccard distance array for the given user handle   
    """
    
    distance_sets = np.full((num_users),1.0) # initialize all distances to 1
    src_index = user_handle - 1 # array is 0-based
    
    for dst_index, dst_row in enumerate(in_array):
        src_row = in_array[src_index]
        dst_row = in_array[dst_index]
        if src_row and dst_row:
            # if either set is empty no point continuing. Otherwise get cardinalities
            intersection_cardinality = len(set.intersection(*[set(src_row), set(dst_row)]))
            union_cardinality = len(set.union(*[set(src_row), set(dst_row)]))
            distance_sets[dst_index] = 1.0 - (
                float(intersection_cardinality)/float(union_cardinality))
    return distance_sets

In [5]:
def calculate_3_axis_distances_for_one_handle(user_handle):
    """
    For a given user_handle, calculate the distance to all other users across all three axes. 
    Assumes Assessments_Obj, Interests_Obj, and Classes_Obj are global. Performs multiple 
    Jaccard Distance caluclations and then uses the Pythagorean Theorem to combine them.
    
    In the end I did not use this method for my final design but am
    maintaining it for the time being.
    
    Inputs - 
    in_array - a 1D array w/ an entry per user. The entry is a set for Jaccard distance calcualtion.
    user_handle - 1-based user handle
    num_users - Number of users
    
    Returns - A Jaccard distance array for the given user handle   
    """
    # note user-handle is 1-based
    assessment_distance = Assessments_Obj.calculate_handle_jaccard_distances(user_handle)
    interests_distance = Interests_Obj.calculate_handle_jaccard_distances(user_handle)
    classes_distance = Classes_Obj.calculate_handle_jaccard_distances(user_handle)
    
    overall_distance = (np.sqrt((assessment_distance ** 2) + (interests_distance ** 2) + \
                                (classes_distance ** 2))) / np.sqrt(3)
    
    return overall_distance

In [6]:
def sql_3_axis_distances_for_one_handle(Assmnts_Obj, Ints_Obj, Cls_Obj, user_handle):
    """
    For a given user_handle, calculate the distance to all other users across all three axes. 
    Assumes Assessments_Obj, Interests_Obj, and Classes_Obj are global. Performs multiple 
    Jaccard Distance caluclations and then uses the Pythagorean Theorem to combine them. 
    Write results to SQLite table.
    
    Inputs - 
    Assmnts_Obj - A member of the Assessments class
    Ints_Obj - A member of the Interests class
    Cls_Obj - A member of the Classes class
    user_handle - 1-based user handle
    
    Returns - Nothing. Results written to SQLite table.   
    """

    # note user-handle is 1-based
    assessment_distance = Assmnts_Obj.calculate_handle_jaccard_distances(user_handle)
    interests_distance = Ints_Obj.calculate_handle_jaccard_distances(user_handle)
    classes_distance = Cls_Obj.calculate_handle_jaccard_distances(user_handle)
    
    overall_distance = (np.sqrt((assessment_distance ** 2) + (interests_distance ** 2) + \
                                (classes_distance ** 2))) / np.sqrt(3)
    conn = sqlite3.connect('dstack.db')
  
    c = conn.cursor()
    for idx, entry in enumerate(overall_distance):
        c.execute("""INSERT INTO distance_matrix VALUES (?, ?, ?)""", 
                  (user_handle, idx + 1, overall_distance[idx],))
        
        
        
    # Save (commit) the changes
    conn.commit()

    conn.close()

    return 

In [7]:
class PS_Data:
    """
    Parent class for Assessments, Intersts, and Classes. Stores dataframe for corresponding csv
    file as well as user_data_sets which stores sets that indicate Assessments, Intersts, and
    Classes for all users. 
    """
    def __init__(self, input_file):
        self.input_file = input_file
        self.data = None
        self.user_list = None
        self.num_users = 0
        self.user_data_sets = None
        self.jaccard_distances = None
        
    def load_data(self):
        """
        Read csv file into a pandas dataframe.
        """
        self.data = pd.read_csv(self.input_file)

    def get_user_handles(self):
        """
        Get all the unique user handles associated with this dataframe
        """
        return self.data.user_handle.unique()
    
    def set_users(self, user_list):
        """
        Use the user_list to define all possible users, not just those associated 
        with the inheriting classes. Allows the set arrays to match in size between
        classes.
        """
        self.user_list = user_list
        self.num_users = len(user_list)
        
    def calculate_all_jaccard_distances(self):
        if self.user_data_sets is None:
            print('calculate_all_jaccard_distances: need to user_data_sets first')
        self.jaccard_distances = jaccard_distances_array(self.user_data_sets, self.num_users)
        
    def calculate_handle_jaccard_distances(self, handle):
        if self.user_data_sets is None:
            print('calculate_all_jaccard_distances: need to user_data_sets first')
        return jaccard_distances_user(self.user_data_sets, handle, self.num_users)

        

In [8]:
class Assessments(PS_Data):
    """
    Class to store assessment related information.
    """
    def __init__(self, input_file):
        PS_Data.__init__(self, input_file)
        
        
        self.load_data()
    
        
    def load_user_data_set(self):
        """
        Convert the dataframe into a user handle based array of sets indicating
        assessment results.
        """
        if self.num_users == 0:
            print('load_user_data_set: Need to load user data to classes first')
            return

        assessment_set = [set() for x in range(len(user_list))]

        for index, row in self.data.iterrows():
            curr_user = assessment_set[row.user_handle-1]
            if (row.user_assessment_score >= 200):
                curr_user.add(row.assessment_tag +'_Expert')
                curr_user.add(row.assessment_tag + '_Proficient')
                curr_user.add(row.assessment_tag +'_Novice')
            elif (row.user_assessment_score >= 100):
                curr_user.add(row.assessment_tag + '_Proficient')
                curr_user.add(row.assessment_tag + '_Novice')
            else:
                curr_user.add(row.assessment_tag + '_Novice')
        self.user_data_sets = assessment_set

In [9]:
class Interests(PS_Data):
    """
    Class to store interests related information.
    """

    def __init__(self, input_file):
        
        PS_Data.__init__(self, input_file)
        
        
        self.load_data()
    
    def load_user_data_set(self):
        """
        Convert the dataframe into a user handle based array of sets indicating
        interests.
        """

        if self.num_users == 0:
            print('load_user_data_set: Need to load user data to classes first')
            return

        interest_set = [set() for x in range(len(user_list))]

        for index, row in self.data.iterrows():
            curr_user = interest_set[row.user_handle-1]
            curr_user.add(row.interest_tag)
        self.user_data_sets = interest_set

In [10]:
class Classes(PS_Data):
    """
    Class to store class-interest related information.
    """

    def __init__(self, input_file, tags_file):
        PS_Data.__init__(self, input_file)
        self.tags_file = tags_file
        self.load_data()
    
    def load_data(self):
        """
        Loading dataframe for Classes is slightly more complex as it also makes use of the 
        tags file.
        """
        self.data_course_tags = pd.read_csv(self.tags_file)
        self.data  = pd.read_csv(self.input_file)

        self.data = self.data.merge(self.data_course_tags[['course_id', 'course_tags']], on=['course_id'])
        self.data.drop('view_date', axis=1, inplace=True)
        self.data.drop('author_handle', axis=1, inplace=True)
        self.data.drop('view_time_seconds', axis=1, inplace=True)
        self.data.drop('course_id', axis=1, inplace=True)
        self.data.drop_duplicates(inplace=True)
        
        
    def load_user_data_set(self):  
        """
        Convert the dataframe into a user handle based array of sets indicating
        class interests.
        """

        course_set = [set() for x in range(len(user_list))]

        for index, row in self.data.iterrows():
            curr_user = course_set[row.user_handle-1]
            course_tag = str(row.course_tags)
            course_tag = course_tag + '_' 
            curr_user.add(course_tag + row.level)
            if row.level == 'Advanced':
                curr_user.add(course_tag + 'Intermediate')
                curr_user.add(course_tag + 'Beginner')
            elif row.level == 'Intermediate':
                curr_user.add(course_tag + 'Beginner')
        self.user_data_sets = course_set




In [11]:
def create_sql_table():
    """
    Connect to the database 'dstack.db' and create the distance_matrix table.
    Delete the table if it already exists.
    
    """
    conn = sqlite3.connect('dstack.db')

    c = conn.cursor()

    # Create table

    c.execute('''DROP TABLE IF EXISTS distance_matrix''')

    c.execute('''CREATE TABLE distance_matrix
             (src_usr integer, dst_usr integer, distance real)''')
    # Save (commit) the changes
    conn.commit()

    conn.close()


In [19]:
# do main stuff here - note there is a main function that repeats this

Assessments_Obj = Assessments('data_files_ml_engineer/user_assessment_scores.csv')
Interests_Obj = Interests('data_files_ml_engineer/user_interests.csv')
Classes_Obj = Classes('data_files_ml_engineer/user_course_views.csv', 'data_files_ml_engineer/course_tags.csv')



Obj_List = [Assessments_Obj,Interests_Obj, Classes_Obj]

user_lists = [obj.get_user_handles() for obj in Obj_List]



user_list = np.unique([item for sublist in user_lists for item in sublist])
[obj.set_users(user_list) for obj in Obj_List]

print('Loading user data sets')
[obj.load_user_data_set() for obj in Obj_List]

create_sql_table()
print('Calculating Jaccard distances across all axes and storing in sql - {0} entries'.format(len(user_list)))
for i in range(1, len(user_list) + 1):
    if (i%25) == 0:
        print (i)
    sql_3_axis_distances_for_one_handle(Assessments_Obj, Interests_Obj, Classes_Obj, i)
print('Done. Data stored via sql.')


25
50
75
100
125
150
175
200
225
250
275
300
325
350
375
400
425
450
475
500
525
550
575
600
625
650
675
700
725
750
775
800
825
850
875
900
925
950
975
1000
1025
1050
1075
1100
1125
1150
1175
1200
1225
1250
1275
1300
1325
1350
1375
1400
1425
1450
1475
1500
1525
1550
1575
1600
1625
1650
1675
1700
1725
1750
1775
1800
1825
1850
1875
1900
1925
1950
1975
2000
2025
2050
2075
2100
2125
2150
2175
2200
2225
2250
2275
2300
2325
2350
2375
2400
2425
2450
2475
2500
2525
2550
2575
2600
2625
2650
2675
2700
2725
2750
2775
2800
2825
2850
2875
2900
2925
2950
2975
3000
3025
3050
3075
3100
3125
3150
3175
3200
3225
3250
3275
3300
3325
3350
3375
3400
3425
3450
3475
3500
3525
3550
3575
3600
3625
3650
3675
3700
3725
3750
3775
3800
3825
3850
3875
3900
3925
3950
3975
4000
4025
4050
4075
4100
4125
4150
4175
4200
4225
4250
4275
4300
4325
4350
4375
4400
4425
4450
4475
4500
4525
4550
4575
4600
4625
4650
4675
4700
4725
4750
4775
4800
4825
4850
4875
4900
4925
4950
4975
5000
5025
5050
5075
5100
5125
5150
5175
5200
52

In [12]:
"""
Main function.
"""

def main():
    Assessments_Obj = Assessments('data_files_ml_engineer/user_assessment_scores.csv')
    Interests_Obj = Interests('data_files_ml_engineer/user_interests.csv')
    Classes_Obj = Classes('data_files_ml_engineer/user_course_views.csv', 'data_files_ml_engineer/course_tags.csv')



    Obj_List = [Assessments_Obj,Interests_Obj, Classes_Obj]

    user_lists = [obj.get_user_handles() for obj in Obj_List]



    user_list = np.unique([item for sublist in user_lists for item in sublist])
    [obj.set_users(user_list) for obj in Obj_List]
    print('Loading user data sets')
    [obj.load_user_data_set() for obj in Obj_List]

    create_sql_table()
    print('Calculating Jaccard distances across all axes and storing in sql - {0} entries'.format(len(user_list)))
    for i in range(1, len(user_list) + 1):
        if (i%25) == 0:
            print (i)
        sql_3_axis_distances_for_one_handle(Assessments_Obj, Interests_Obj, Classes_Obj, i)
    print('Done. Data stored via sql.')
    
    
if __name__ == '__main__':
    main()

Loading user data sets


NameError: name 'user_list' is not defined

In [24]:
conn = sqlite3.connect('dstack.db')
  
c = conn.cursor()
c.execute("""SELECT dst_usr, distance FROM distance_matrix where src_usr = 9999 and src_usr != dst_usr ORDER BY distance LIMIT 10""")
        
rows = c.fetchall()
        
        
# Save (commit) the changes
conn.commit()

conn.close()

user_dict =  {k:v for k, v in rows}




In [25]:
rows

[(1562, 0.0),
 (5238, 0.0),
 (6303, 0.0),
 (4484, 0.5),
 (4432, 0.5196152422706632),
 (906, 0.5204164998665332),
 (2200, 0.5443310539518175),
 (3605, 0.5446711546122731),
 (5071, 0.5608545472127794),
 (453, 0.5773502691896258)]

In [26]:
user_dict

{1562: 0.0,
 5238: 0.0,
 6303: 0.0,
 4484: 0.5,
 4432: 0.5196152422706632,
 906: 0.5204164998665332,
 2200: 0.5443310539518175,
 3605: 0.5446711546122731,
 5071: 0.5608545472127794,
 453: 0.5773502691896258}